# Grouping with Pivot Tables

A pivot table aggregates data between the intersection of the unique values of two (or more) columns of your data. In the pivot table below, the two columns are `race` and `sex`. All pivot tables must aggregate some other column of data. Here, the salary is averaged. There are 5 unique races and 2 unique values for sex. The pivot table shows the mean of salary for each possible combination. Having the data in this structure, can make it easier to read and make comparisons.

![1]

[1]: images/pivot_table_example.png

### Creating a simple pivot table in pandas - four components

There are four components to a basic pivot table in pandas.

* One vertical grouping column
* One horizontal grouping column
* One aggregating column
* One aggregating function

In the example above, the two grouping columns are `race` and `sex`. The aggregating column is `salary` and the aggregating function is `mean`.

## Creating pivot tables with pandas

Let's read in the employee dataset and use it to recreate the pivot table above in pandas.

In [1]:
import pandas as pd
emp = pd.read_csv('../data/employee.csv', parse_dates=['hire_date'])
emp.head(3)

,dept,title,hire_date,salary,sex,race
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black


To use the `pivot_table` method, set the `index`, `columns`, `values`, and `aggfunc` parameters:

* `index` - vertical grouping column
* `columns` - horizontal grouping column
* `values` - aggregating column
* `aggfunc` - aggregating function (defaulted to the mean)

In [2]:
emp.pivot_table(index='race', columns='sex', values='salary', aggfunc='mean')

sex,Female,Male
race,,
Asian,65846.027190,65071.505725
Black,52416.790860,52154.353471
Hispanic,48835.416765,57637.428134
Native American,48767.227179,61672.815192
White,66411.819615,66653.901622


### Round results and convert to integer to reduce noise

The above DataFrame has several excess decimal values that aren't important. Rounding to the nearest thousand and changing the data type to be integers reduces this noise.

In [3]:
emp.pivot_table(index='race', columns='sex', 
                values='salary', aggfunc='mean').round(-3).astype('int64')

sex,Female,Male
race,,
Asian,66000,65000
Black,52000,52000
Hispanic,49000,58000
Native American,49000,62000
White,66000,67000


### Easily compare female vs male salary

Pivot tables make comparisons between groups easy. In this instance, the difference between female and male average salary is easily seen.

### Column and index labels are sorted

Notice that the labels for each of the index and columns of a pivot table come from the unique values of the grouping columns. They are also sorted in alphabetical order. The intersection of each label is where the aggregated data appears.

## Comparison to `groupby`

The `pivot_table` method is very similar to a `groupby` aggregation. Both are capable of producing the exact same results. Below, we replicate the results of our pivot table with `groupby`.

In [4]:
(emp.groupby(['race', 'sex'])
    .agg(mean_salary=('salary', 'mean'))
    .round(-3)
    .astype('int64'))

mean_salary
race            sex                
Asian           Female        66000
                Male          65000
Black           Female        52000
                Male          52000
Hispanic        Female        49000
                Male          58000
Native American Female        49000
                Male          62000
White           Female        66000
                Male          67000

### Data is more difficult to make comparisons

This call to `groupby` produced the exact same result as the pivot table but in a different shape. Having all of our data in a vertical column makes it difficult to make comparisons.

### Wide vs long data

Pivot tables produce **wide** data, with new columns for each unique value of one of the grouping columns. Wide data is typically easier to read and make decisions with. The `groupby` method returns **long** data with the results of each group in a single column, making it more difficult to make comparisons. This type of data is generally easier to continue analyzing with further commands.

### The default aggregation is `mean`

By default, the `aggfunc` parameter is set to 'mean'. But, even if you are using the mean as your aggregation function, I advise that you explicitly state it in your call to `pivot_table` so that it's clear what you are doing.

### All aggregation strings are available for `pivot_table`

The same aggregation strings ('min', 'max', 'mean', etc...) are available to a `pivot_table` as they are with `groupby`. Here we find the max salary for the same groups.

In [5]:
emp.pivot_table(index='race', columns='sex', values='salary', aggfunc='max')

sex,Female,Male
race,,
Asian,342784.0,342784.0
Black,342784.0,342784.0
Hispanic,180000.0,342784.0
Native American,99784.0,121548.0
White,342784.0,342784.0


### Where is the 'pivoting'?

Microsoft Excel is well-known for its pivot tables that are created by dragging and dropping different columns into different boxes, 'pivoting' the data around. With pandas, you'll have to change the parameter values and call the `pivot_table` method again in order to get the same effect. Let's pivot the table by putting sex along the index and race along the columns.

In [ ]:
emp.pivot_table(index='sex', columns='race', values='salary', aggfunc='max')


race,Asian,Black,Hispanic,Native American,White
sex,,,,,
Female,342784.0,342784.0,180000.0,99784.0,342784.0
Male,342784.0,342784.0,342784.0,121548.0,342784.0


## Styling pivot tables

You can style any DataFrame by changing the text color, background color, font, and several other items with the `style` accessor. It works similarly to `str`, `dt`, and `cat` accessors in that it gives you access to style-only methods through dot notation. [Visit the documentation][1] for descriptions on all of the methods. Let's create a pivot table computing the mean salary by department and race and assign the result to a variable.

[1]: http://pandas.pydata.org/pandas-docs/stable/style.html

In [7]:
dept_race_mean = (emp.pivot_table(index='dept', columns='race', 
                                 values='salary', aggfunc='mean')
                     .round(-3).astype('int64'))
dept_race_mean

race,Asian,Black,Hispanic,Native American,White
dept,,,,,
Fire,69000,60000,57000,62000,62000
Health & Human Services,64000,57000,47000,45000,66000
Houston Airport System,53000,51000,46000,60000,69000
Houston Public Works,68000,45000,49000,52000,66000
Library,40000,40000,40000,30000,48000
Other,72000,59000,53000,57000,75000
Parks & Recreation,40000,35000,36000,39000,45000
Police,67000,62000,65000,69000,71000
Solid Waste Management,62000,43000,43000,31000,48000


### Highlighting the maximum value

The `style` accessor's `highlight_max` method highlights the maximum value in each column or row. By default, it highlights the maximum value of each column, just like most other pandas methods.

In [9]:
dept_race_mean.style.highlight_max()

race,Asian,Black,Hispanic,Native American,White
dept,,,,,
Fire,69000,60000,57000,62000,62000
Health & Human Services,64000,57000,47000,45000,66000
Houston Airport System,53000,51000,46000,60000,69000
Houston Public Works,68000,45000,49000,52000,66000
Library,40000,40000,40000,30000,48000
Other,72000,59000,53000,57000,75000
Parks & Recreation,40000,35000,36000,39000,45000
Police,67000,62000,65000,69000,71000
Solid Waste Management,62000,43000,43000,31000,48000


### Change direction with `axis`

You can highlight the max of each row by setting the `axis` parameter to `'columns'` or 1. A single cell of the entire table maximum can be highlighted by setting `axis` to `None` (not shown).

In [10]:
dept_race_mean.style.highlight_max(axis='columns')

race,Asian,Black,Hispanic,Native American,White
dept,,,,,
Fire,69000,60000,57000,62000,62000
Health & Human Services,64000,57000,47000,45000,66000
Houston Airport System,53000,51000,46000,60000,69000
Houston Public Works,68000,45000,49000,52000,66000
Library,40000,40000,40000,30000,48000
Other,72000,59000,53000,57000,75000
Parks & Recreation,40000,35000,36000,39000,45000
Police,67000,62000,65000,69000,71000
Solid Waste Management,62000,43000,43000,31000,48000


### Background color gradients

Use `background_gradient` to color the background based on the value of the cell. You can change the colors by choosing a [Matplotlib colormap][1].

[1]: https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [14]:
dept_race_mean.style.background_gradient(cmap='Oranges')

race,Asian,Black,Hispanic,Native American,White
dept,,,,,
Fire,69000,60000,57000,62000,62000
Health & Human Services,64000,57000,47000,45000,66000
Houston Airport System,53000,51000,46000,60000,69000
Houston Public Works,68000,45000,49000,52000,66000
Library,40000,40000,40000,30000,48000
Other,72000,59000,53000,57000,75000
Parks & Recreation,40000,35000,36000,39000,45000
Police,67000,62000,65000,69000,71000
Solid Waste Management,62000,43000,43000,31000,48000


### Adding commas to numbers

Make your data easier to read by inserting commas into large numbers with the `format` method. You must know how to use the [string format specification][0] from core Python. To add commas, use the string `'{:,.0f}'`. If you are unfamiliar with format specification, use the link provided. But, as a quick overview, the actual specification is what comes after the colon and does not include the curly braces. Here, it is `',.0f'`. The comma is the digit separator. The `.0` represents the numbers after the decimal, which is 0 in this case. The character `'f'` is the 'type' and stands for fixed-point notation. If you wanted to include two decimal places, you would use the format specification `'{:,.2f}'`.

[0]: https://docs.python.org/3/library/string.html#formatspec

In [15]:
dept_race_mean.style.format('{:,.0f}')

race,Asian,Black,Hispanic,Native American,White
dept,,,,,
Fire,"69,000","60,000","57,000","62,000","62,000"
Health & Human Services,"64,000","57,000","47,000","45,000","66,000"
Houston Airport System,"53,000","51,000","46,000","60,000","69,000"
Houston Public Works,"68,000","45,000","49,000","52,000","66,000"
Library,"40,000","40,000","40,000","30,000","48,000"
Other,"72,000","59,000","53,000","57,000","75,000"
Parks & Recreation,"40,000","35,000","36,000","39,000","45,000"
Police,"67,000","62,000","65,000","69,000","71,000"
Solid Waste Management,"62,000","43,000","43,000","31,000","48,000"


### Chaining style methods

You can chain together multiple style methods to your DataFrame. Before doing so, it's important to know that the returned object from a call to one of the style methods is NOT a DataFrame. It's a `Styler` object. We verify this below.

In [16]:
df_styled = dept_race_mean.style.format('{:,.0f}')
type(df_styled)

pandas.io.formats.style.Styler

You cannot use this object as a normal DataFrame. If you try and call a normal DataFrame method such as `mean`, you'll get an error.

In [17]:
df_styled.mean()

AttributeError: 'Styler' object has no attribute 'mean'

You can retrieve the original (unstyled data) with the `data` attribute. Here, we verify it is the same object as the pivot table we calculated above.

In [19]:
df_styled.data

race,Asian,Black,Hispanic,Native American,White
dept,,,,,
Fire,69000,60000,57000,62000,62000
Health & Human Services,64000,57000,47000,45000,66000
Houston Airport System,53000,51000,46000,60000,69000
Houston Public Works,68000,45000,49000,52000,66000
Library,40000,40000,40000,30000,48000
Other,72000,59000,53000,57000,75000
Parks & Recreation,40000,35000,36000,39000,45000
Police,67000,62000,65000,69000,71000
Solid Waste Management,62000,43000,43000,31000,48000


In [18]:
df_styled.data is dept_race_mean

True

The benefit of having this Styler object is that you can chain one style method after another without referencing the `style` attribute again. Here, we add commas and highlight the maximum and minimum value of each column different colors.

In [20]:
(dept_race_mean.style.format('{:,.0f}')
                     .highlight_max(color='yellow')
                     .highlight_min(color='lightblue'))

race,Asian,Black,Hispanic,Native American,White
dept,,,,,
Fire,"69,000","60,000","57,000","62,000","62,000"
Health & Human Services,"64,000","57,000","47,000","45,000","66,000"
Houston Airport System,"53,000","51,000","46,000","60,000","69,000"
Houston Public Works,"68,000","45,000","49,000","52,000","66,000"
Library,"40,000","40,000","40,000","30,000","48,000"
Other,"72,000","59,000","53,000","57,000","75,000"
Parks & Recreation,"40,000","35,000","36,000","39,000","45,000"
Police,"67,000","62,000","65,000","69,000","71,000"
Solid Waste Management,"62,000","43,000","43,000","31,000","48,000"


## Getting the size of each group

You can use the `pivot_table` method to get the total number of occurrences of each of the grouping columns. When doing so, it is not necessary to use an aggregating column (the `values` parameter). pandas knows that the size of the group is independent of what it is aggregating, so it does not require you to provide it. Here, we get the size of each unique combination of department and sex.

In [21]:
emp.pivot_table(index='dept', columns='race', aggfunc='size')

race,Asian,Black,Hispanic,Native American,White
dept,,,,,
Fire,80,699,1159,36,2399
Health & Human Services,140,668,399,11,132
Houston Airport System,123,427,317,9,337
Houston Public Works,377,2229,820,24,735
Library,47,223,139,4,150
Other,256,1515,898,18,680
Parks & Recreation,36,641,329,4,139
Police,479,1852,2010,39,3189
Solid Waste Management,9,407,77,1,18


## Add margins to get row and column totals

The optional parameter `margins` can be set to `True` to add one additional row and column to the pivot table which calculates the same aggregate function to the entire row or column. In the following pivot table, the average salary for all fire department employees is 61,000, and the average salary for all Hispanic employees is 55,000.

In [34]:
(emp.pivot_table(index='dept', columns='race', values='salary', 
                 aggfunc='mean', margins=True)
    .round(-3)
    .style.format('{:,.0f}'))

race,Asian,Black,Hispanic,Native American,White,All
dept,,,,,,
Fire,"69,000","60,000","57,000","62,000","62,000","61,000"
Health & Human Services,"64,000","57,000","47,000","45,000","66,000","55,000"
Houston Airport System,"53,000","51,000","46,000","60,000","69,000","55,000"
Houston Public Works,"68,000","45,000","49,000","52,000","66,000","51,000"
Library,"40,000","40,000","40,000","30,000","48,000","42,000"
Other,"72,000","59,000","53,000","57,000","75,000","61,000"
Parks & Recreation,"40,000","35,000","36,000","39,000","45,000","37,000"
Police,"67,000","62,000","65,000","69,000","71,000","67,000"
Solid Waste Management,"62,000","43,000","43,000","31,000","48,000","44,000"


In [35]:
(emp.pivot_table(index='dept', columns='race', values='salary', 
                 aggfunc='mean', margins=True)
    .round(-3)
    .drop(columns='All')
    .style.format('{:,.0f}'))

race,Asian,Black,Hispanic,Native American,White
dept,,,,,
Fire,"69,000","60,000","57,000","62,000","62,000"
Health & Human Services,"64,000","57,000","47,000","45,000","66,000"
Houston Airport System,"53,000","51,000","46,000","60,000","69,000"
Houston Public Works,"68,000","45,000","49,000","52,000","66,000"
Library,"40,000","40,000","40,000","30,000","48,000"
Other,"72,000","59,000","53,000","57,000","75,000"
Parks & Recreation,"40,000","35,000","36,000","39,000","45,000"
Police,"67,000","62,000","65,000","69,000","71,000"
Solid Waste Management,"62,000","43,000","43,000","31,000","48,000"


The average salary of all employees is given by the bottom right value. Let's verify this is correct by computing the average manually.

In [25]:
emp['salary'].mean()

np.float64(58206.76157092714)

## Non-standard pivot tables

There are many different kinds of pivot tables that we can create besides one with exactly two grouping columns, one aggregating column, and one aggregating function. I am calling such pivot tables 'non-standard', though this is just to help differentiate them from the previous ones created above.

### A single grouping column

It is possible to have just a single grouping column in a pivot table. Here, we find the average salary for each department.

In [36]:
emp.pivot_table(index='dept', values='salary', aggfunc='mean').round(-3)

,salary
dept,
Fire,61000.0
Health & Human Services,55000.0
Houston Airport System,55000.0
Houston Public Works,51000.0
Library,42000.0
Other,61000.0
Parks & Recreation,37000.0
Police,67000.0
Solid Waste Management,44000.0


When using a single grouping column, the result will be the exact same as a `groupby` aggregation. Using `groupby` has the advantage of being able to rename the resulting aggregate column.

In [37]:
emp.groupby('dept').agg(average_salary=('salary', 'mean')).round(-3)

,average_salary
dept,
Fire,61000.0
Health & Human Services,55000.0
Houston Airport System,55000.0
Houston Public Works,51000.0
Library,42000.0
Other,61000.0
Parks & Recreation,37000.0
Police,67000.0
Solid Waste Management,44000.0


You can pivot this one column table by using the `columns` parameter.

In [38]:
emp.pivot_table(columns='dept', values='salary', aggfunc='mean').round(-3)

dept,Fire,Health & Human Services,Houston Airport System,Houston Public Works,Library,Other,Parks & Recreation,Police,Solid Waste Management
salary,61000.0,55000.0,55000.0,51000.0,42000.0,61000.0,37000.0,67000.0,44000.0


### More than two grouping columns

You can use any number of grouping columns when creating pivot tables. Use a list to contain the columns you want for the rows or columns. Here we find the average salary by department, sex, and race. The unique combinations of department and sex are placed in the index. The resulting DataFrame has a multi-level index.

In [44]:
(emp.pivot_table(index=['dept', 'sex'], columns='race', 
                 values='salary', aggfunc='max')
    .round(-3)
    .head(10)
    .style.format('{:,.0f}'))

We can pivot this so that the result so that there is a multi-level column index.

In [45]:
(emp.pivot_table(index='dept', columns=['race', 'sex'],
                 values='salary', aggfunc='max')
    .round(-3)
    .style.format('{:,.0f}'))

### Keep the multi-level index with pivot tables

I advise that you keep the multiple levels when using a pivot table (if you happen to produce them). This is the opposite advice that I gave when grouping. The reasoning is that pivot tables are much more likely to be a final product - something that you use in a presentation or a report and won't be doing further analysis on. Therefore, you won't have to handle the multi-level index or columns. After doing a groupby, it is more likely that you'll be running other pandas commands, and doing so is much easier with a normal single level index.

### Multiple aggregating columns

It is possible to aggregate more than one column as well. Because the employee dataset only has one main aggregating column, we will add a column for years of experience. The data was pulled in 2019, so we will subtract the year hired from it to get the approximate years of experience.

In [46]:
emp['experience'] = 2019 - emp['hire_date'].dt.year
emp.head(3)

,dept,title,hire_date,salary,sex,race,experience
0,Police,POLICE SERGEANT,2001-12-03,87545.38,Male,White,18
1,Other,ASSISTANT CITY ATTORNEY II,2010-11-15,82182.00,Male,Hispanic,9
2,Houston Public Works,SENIOR SLUDGE PROCESSOR,2006-01-09,49275.00,Male,Black,13


Let's find the average salary and experience for every department and race. Notice that the columns are now multiple levels. Because of this, it becomes harder to use methods like `round` which you might need to specify different decimals for different columns. We compromise here and format our data to a single decimal place. The margins for each aggregating column are also there.

In [47]:
emp.pivot_table(index='dept', columns='race', 
                values=['salary', 'experience'],
                aggfunc='mean', margins=True).style.format('{:,.1f}')

### Multiple aggregating functions

All components of a pivot table are capable of taking multiple values, including the aggregating functions. Here, we find the minimum, maximum, and average salary for each department and sex.

In [48]:
(emp.pivot_table(index='dept', columns='sex', values='salary',
                aggfunc=['min', 'max', 'mean'], margins=True)
    .round(-3)
    .style.format('{:,.0f}'))

### Reducing readability of a pivot table

Pivot tables with more than two grouping columns, or multiple aggregating columns and aggregating functions become less readable as the amount of data displayed can be immense. Here, we find the minimum, maximum, and average salary and experience for all combinations of department, sex, and race. Only the first few rows and columns are output.

In [49]:
df_messy_pivot = emp.pivot_table(index=['dept', 'sex'], columns='race', 
                                 values=['salary', 'experience'],
                                 aggfunc=['min', 'max', 'mean'], margins=True)
df_messy_pivot.iloc[:6, :10]

min                                 \
                               experience                                  
race                                Asian Black Hispanic Native American   
dept                    sex                                                
Fire                    Female        1.0   1.0      1.0             4.0   
                        Male          1.0   1.0      1.0             1.0   
Health & Human Services Female        1.0   1.0      1.0             4.0   
                        Male          1.0   1.0      1.0             2.0   
Houston Airport System  Female        1.0   1.0      1.0             2.0   
                        Male          1.0   1.0      1.0             2.0   

                                                                     \
                                           salary                     
race                           White All    Asian    Black Hispanic   
dept                    sex                                           
Fire                    Female   1.0   1  39104.0  16411.0  28024.0   
                        Male     1.0   1  28024.0  28024.0  26000.0   
Health & Human Services Female   1.0   1  30888.0  25064.0  27560.0   
                        Male     1.0   1  35714.0  25064.0  25813.0   
Houston Airport System  Female   1.0   1  26894.0  26125.0  26125.0   
                        Male     1.0   1  26915.0  26125.0  26894.0   

                                                
                                                
race                           Native American  
dept                    sex                     
Fire                    Female         48189.7  
                        Male           28024.0  
Health & Human Services Female         33010.0  
                        Male           42848.0  
Houston Airport System  Female         29058.0  
                        Male           39790.0

These large tables can be useful, but the results are often better presented as multiple different pivot tables than a single one. This pivot table has 19 rows and 36 columns with two levels in the index and three levels in the columns. It won't easily fit on a single screen.

In [50]:
df_messy_pivot.shape

(19, 36)

## Exercises

Execute the following cell to read in the flights dataset and insert columns for the day and month name. Use it for the following exercises.

In [ ]:
import pandas as pd
flights = pd.read_csv('../data/flights.csv', parse_dates=['date'])
flights.insert(1, 'day_of_week', flights['date'].dt.day_name())
flights.insert(2, 'month', flights['date'].dt.month_name())
flights.head(3)

,date,day_of_week,month,airline,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-01-01,Monday,January,UA,LAS,IAH,100,547,0,134.0,1222.0,0,0,0,0,0
1,2018-01-01,Monday,January,WN,DEN,PHX,515,720,0,91.0,602.0,0,0,0,0,0
2,2018-01-01,Monday,January,B6,JFK,BOS,550,657,0,39.0,187.0,0,83,8,0,0


In [57]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_order_sorted = pd.CategoricalDtype(day_order, ordered=True)
flights['day_of_week'] = flights['day_of_week'].astype(day_order_sorted)

In [81]:
month_order = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]

# Create an ordered categorical type
month_dtype = pd.CategoricalDtype(categories=month_order, ordered=True)

# Apply it to your column
flights['month'] = flights['month'].astype(month_dtype)

### Exercise 1

<span style="color:green; font-size:16px">What is the average carrier delay for each day of the week for each airline? Highlight the worst day of the week for each airline.</span>

In [65]:
(flights.pivot_table(index='airline',columns='day_of_week', values='carrier_delay', aggfunc='mean')
        .round(1)
        .style
        .highlight_max(axis=1,color='green')
        .format('{:.1f}')
        )

C:\Users\nealv\AppData\Local\Temp\ipykernel_29488\2784208990.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  (flights.pivot_table(index='airline',columns='day_of_week', values='carrier_delay', aggfunc='mean')


day_of_week,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
airline,,,,,,,
9E,4.7,3.0,4.2,7.7,3.4,1.8,3.3
AA,3.7,4.1,2.8,4.9,3.9,3.8,3.8
AS,1.5,2.6,2.1,2.9,3.0,3.6,3.7
B6,4.0,6.2,4.0,5.1,7.0,5.5,5.1
DL,2.6,3.3,3.3,3.9,3.5,3.0,3.6
EV,8.5,6.0,1.7,0.0,5.2,2.4,7.2
F9,3.6,6.6,1.8,7.8,7.9,9.5,4.1
MQ,4.2,1.5,2.3,4.2,2.1,0.0,5.4
NK,1.5,2.5,1.2,2.2,2.2,5.8,1.7


### Exercise 2

<span style="color:green; font-size:16px">Use a pivot table to find the total number of canceled flights for each origin airport and airline. Place the airlines in the columns. Use the result to find the origin airport with the most cancelled flights for each airline. Also return this maximum number of cancelled flights.</span>

In [66]:
flights.head(3)

,date,day_of_week,month,airline,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-01-01,Monday,January,UA,LAS,IAH,100,547,0,134.0,1222.0,0,0,0,0,0
1,2018-01-01,Monday,January,WN,DEN,PHX,515,720,0,91.0,602.0,0,0,0,0,0
2,2018-01-01,Monday,January,B6,JFK,BOS,550,657,0,39.0,187.0,0,83,8,0,0


In [78]:
flights.pivot_table(index='origin',columns='airline',values='cancelled',aggfunc='sum').agg(['max','idxmax'], axis=1)

,max,idxmax
origin,,
ATL,19.0,DL
BOS,41.0,AA
CLT,33.0,AA
DCA,31.0,YX
DEN,10.0,UA
DFW,33.0,AA
DTW,8.0,DL
EWR,27.0,UA
IAH,7.0,AA


### Exercise 3

<span style="color:green; font-size:16px">Find the total distance flown for each airline for each month. Highlight the month with the most number of miles flown and use the style `format` method to put commas in the numbers so that they are easier to read.</span>

In [79]:
flights.head(3)

,date,day_of_week,month,airline,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-01-01,Monday,January,UA,LAS,IAH,100,547,0,134.0,1222.0,0,0,0,0,0
1,2018-01-01,Monday,January,WN,DEN,PHX,515,720,0,91.0,602.0,0,0,0,0,0
2,2018-01-01,Monday,January,B6,JFK,BOS,550,657,0,39.0,187.0,0,83,8,0,0


In [90]:
(flights.pivot_table(index='airline',columns='month',values='distance', aggfunc='sum', observed=False)
        .style
        .format('{:,.2f}')
        .highlight_max(axis=1, color='green')

)

month,January,February,March,April,May,June,July,August,September,October,November,December
airline,,,,,,,,,,,,
9E,"47,230.00","51,784.00","61,460.00","54,592.00","42,423.00","50,421.00","53,868.00","62,216.00","45,745.00","48,106.00","42,275.00","46,032.00"
AA,"1,473,883.00","1,371,620.00","1,528,361.00","1,586,655.00","1,545,453.00","1,619,325.00","1,669,007.00","1,649,436.00","1,482,841.00","1,588,285.00","1,409,540.00","1,444,276.00"
AS,"195,553.00","201,275.00","199,288.00","454,146.00","495,090.00","496,358.00","455,061.00","451,512.00","429,045.00","409,479.00","391,304.00","399,787.00"
B6,"385,517.00","348,189.00","382,666.00","352,234.00","410,877.00","443,151.00","478,230.00","404,458.00","384,008.00","425,712.00","384,038.00","427,097.00"
DL,"1,017,440.00","997,216.00","1,215,516.00","1,265,266.00","1,253,361.00","1,292,928.00","1,396,697.00","1,315,865.00","1,173,359.00","1,214,950.00","1,100,681.00","1,160,997.00"
EV,"10,186.00","11,854.00","4,511.00","6,847.00","3,569.00","5,926.00",927.00,"1,194.00",995.00,"2,587.00","1,592.00","3,933.00"
F9,"118,067.00","97,879.00","80,444.00","117,439.00","78,807.00","116,116.00","84,417.00","97,777.00","89,938.00","105,833.00","110,423.00","97,846.00"
MQ,"15,170.00","17,539.00","13,349.00","13,060.00","13,656.00","15,310.00","20,057.00","15,787.00","14,767.00","16,884.00","15,559.00","14,057.00"
NK,"249,461.00","219,678.00","228,829.00","250,683.00","261,421.00","318,648.00","273,963.00","270,894.00","235,754.00","253,692.00","266,838.00","232,613.00"


### Exercise 4

<span style="color:green; font-size:16px">Create a pivot table that shows the number of flights flown for every day of the week for every month.</span>

In [93]:
flights.head(3)

,date,day_of_week,month,airline,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-01-01,Monday,January,UA,LAS,IAH,100,547,0,134.0,1222.0,0,0,0,0,0
1,2018-01-01,Monday,January,WN,DEN,PHX,515,720,0,91.0,602.0,0,0,0,0,0
2,2018-01-01,Monday,January,B6,JFK,BOS,550,657,0,39.0,187.0,0,83,8,0,0


In [98]:
flights_not_cancelled = flights.query('cancelled == 0')

flights_not_cancelled.pivot_table(index='month',columns='day_of_week', aggfunc='size',observed=False)

day_of_week,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
month,,,,,,,
January,842,801,812,675,652,535,691
February,718,705,709,750,732,541,629
March,730,633,665,861,920,699,686
April,928,749,743,815,783,635,886
May,791,892,877,1046,761,597,724
June,785,759,817,831,997,809,799
July,919,980,766,754,784,669,916
August,766,732,951,962,975,573,765
September,732,715,756,759,733,735,895


### Exercise 5

<span style="color:green; font-size:16px">In exercise 4, the months and days of week are ordered alphabetically. It would be better if these values were ordered chronologically. Can you return a result that has both groups in the correct order. Use Monday as the first day of the week.</span>

In [99]:

flights_not_cancelled.pivot_table(index='month',columns='day_of_week', aggfunc='size',observed=False)

day_of_week,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
month,,,,,,,
January,842,801,812,675,652,535,691
February,718,705,709,750,732,541,629
March,730,633,665,861,920,699,686
April,928,749,743,815,783,635,886
May,791,892,877,1046,761,597,724
June,785,759,817,831,997,809,799
July,919,980,766,754,784,669,916
August,766,732,951,962,975,573,765
September,732,715,756,759,733,735,895


### Exercise 6

<span style="color:green; font-size:16px">Create a new column in the flights dataset called `'dep_time_hour'` and set it equal to the hour (this will be an integer 0 through 23) of the flight. Find the average carrier delay for every month and `dep_time_hour`. Place the month in the columns.</span>

In [100]:
flights.head(3)

,date,day_of_week,month,airline,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-01-01,Monday,January,UA,LAS,IAH,100,547,0,134.0,1222.0,0,0,0,0,0
1,2018-01-01,Monday,January,WN,DEN,PHX,515,720,0,91.0,602.0,0,0,0,0,0
2,2018-01-01,Monday,January,B6,JFK,BOS,550,657,0,39.0,187.0,0,83,8,0,0


In [111]:
flights['dep_time_hour'] = flights['dep_time'].floordiv(100)

flights.pivot_table(index='dep_time_hour',columns='month',values='carrier_delay',aggfunc='mean',observed=False).round(1).style.highlight_max(color='green',axis=1).format('{:.1f}')

month,January,February,March,April,May,June,July,August,September,October,November,December
dep_time_hour,,,,,,,,,,,,
0,4.6,1.8,3.6,4.3,0.8,0.4,1.8,2.2,4.7,1.0,1.9,4.2
1,0.0,0.9,0.0,0.0,0.0,2.8,1.6,2.5,0.0,2.1,0.0,0.0
2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,nan
5,1.4,2.3,2.0,1.2,7.3,5.1,2.5,2.9,7.6,0.7,2.5,1.9
6,6.9,3.2,3.2,2.5,2.3,2.5,6.4,2.6,1.8,2.7,3.9,1.7
7,1.4,2.1,2.7,2.1,5.6,3.7,3.8,2.8,2.2,2.1,2.4,4.2
8,2.3,3.4,3.4,2.9,6.1,3.6,6.2,6.2,5.1,1.7,3.2,4.0
9,5.8,3.7,2.0,2.3,2.5,3.6,2.4,2.3,1.6,3.0,4.4,4.2


### Exercise 7

<span style="color:green; font-size:16px">Use both `groupby` and `pivot_table` to compute the average and median distance flown by day of the week.</span>

In [112]:
flights.head(3)

,date,day_of_week,month,airline,origin,dest,dep_time,arr_time,cancelled,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,dep_time_hour
0,2018-01-01,Monday,January,UA,LAS,IAH,100,547,0,134.0,1222.0,0,0,0,0,0,1
1,2018-01-01,Monday,January,WN,DEN,PHX,515,720,0,91.0,602.0,0,0,0,0,0,5
2,2018-01-01,Monday,January,B6,JFK,BOS,550,657,0,39.0,187.0,0,83,8,0,0,5


In [114]:
flights.pivot_table(index='day_of_week', values='distance', aggfunc=['mean','median'], observed=False)

,mean,median
,distance,distance
day_of_week,,
Monday,1071.351038,912.0
Tuesday,1051.831141,888.0
Wednesday,1052.546290,868.0
Thursday,1066.285073,907.0
Friday,1050.703404,868.0
Saturday,1107.159728,937.0
Sunday,1092.702657,925.0


In [117]:
flights.groupby('day_of_week', observed=False).agg(avg_dist=('distance','mean'), med_distance=('distance','median'))

,avg_dist,med_distance
day_of_week,,
Monday,1071.351038,912.0
Tuesday,1051.831141,888.0
Wednesday,1052.546290,868.0
Thursday,1066.285073,907.0
Friday,1050.703404,868.0
Saturday,1107.159728,937.0
Sunday,1092.702657,925.0
